In [ ]:
from k3d.helpers import download
from pyunpack import Archive
import os
from IPython.display import IFrame

filename = download('http://www.semantic3d.net/data/point-clouds/testing1/stgallencathedral_station1_intensity_rgb.7z')
Archive(filename).extractall('./')

In [ ]:
import csv
import numpy as np 

data = None

with open(filename.replace('.7z', '.txt'), mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ')    
    data = np.array(list(csv_reader), dtype=np.float32)

# compute color in hex format
data[:, 4] = np.sum(data[:, 4:7].astype(np.uint32) * np.array([1, 256, 256 ** 2]), axis=1)    
data = data[:, 0:5]

In [ ]:
positions = data[::10, 0:3]
colors1 = data[::10, 4].astype(np.uint32)
colors2 = np.clip(colors1.view(np.uint8) * 1.5, 0, 255).astype(np.uint8).view('uint32')

In [ ]:
positions.shape, colors1.shape, colors2.shape

In [ ]:
cam_pos = [5.251483149143791,
 -7.92683507646606,
 3.144285796928443,
 -2.470283607444292,
 3.6558150584160503,
 2.3721091212696286,
 0,
 0,
 1]

In [ ]:
import time
import json
import k3d
import ipywidgets as widgets

plot = k3d.plot()
plot += k3d.points(positions, colors1, point_size=0.2, shader="flat") 
plot += k3d.points(positions, colors2, point_size=0.2, shader="flat") 
plot.display()

In [ ]:
plot.camera = cam_pos

In [ ]:
with open("point_cloud_full.html", "w") as f:
    f.write(plot.get_snapshot(additional_js_code="""
    let json = K3DInstance.getWorld().ObjectsListJson;    
    let active = Object.keys(json)[0];
    
    setInterval(() => {
        let newActive;
        
        Object.keys(json).forEach(function (id) {
          if (active === id) {
            json[id].opacity = 0;
            K3DInstance.reload(json[id], {opacity: 0});            
          } else {
            json[id].opacity = 1;
            K3DInstance.reload(json[id], {opacity: 1});
            newActive = id;
          }                    
        });
        
        active = newActive;
      
    }, 2000);
    
    
    """))

In [ ]:
# size in html
os.stat("point_cloud_full.html").st_size / 1024 / 1024 # size in MB

In [ ]:
IFrame('point_cloud_full.html', width=900, height=350)

# Advanced approach

In [ ]:
import time
import json
import k3d
import ipywidgets as widgets

plot2 = k3d.plot()

positions_placeholder = positions.copy()
positions_placeholder.fill(0.0) # filling with one value will boost compression ratio

plot2 += k3d.points(positions, colors1, point_size=0.2, shader="flat", name="main") 
plot2 += k3d.points(positions_placeholder, colors2, point_size=0.2, shader="flat", name="second") 

plot2.display()

In [ ]:
plot2.camera = cam_pos

In [ ]:
with open("point_cloud_advanced.html", "w") as f:
    f.write(plot2.get_snapshot(additional_js_code="""
    let json = K3DInstance.getWorld().ObjectsListJson;    
    let active = Object.keys(json)[0];
    let positions = null;
    
    // search for main and save positions from it
    Object.keys(json).forEach(function (id) {
      if (json[id].name === 'main') {
        positions = json[id].positions.data;
      }
    });
    
    // search for second and update positions
    Object.keys(json).forEach(function (id) {
      if (json[id].name === 'second') {
        json[id].positions.data.set(positions);
        K3DInstance.reload(json[id], {positions: json[id].positions}); 
      }
    });
    
    
    // like before
    
    setInterval(() => {
        let json = K3DInstance.getWorld().ObjectsListJson;
        let newActive;
        
        Object.keys(json).forEach(function (id) {
          if (active === id) {
            json[id].opacity = 0;
            K3DInstance.reload(json[id], {opacity: 0});            
          } else {
            json[id].opacity = 1;
            K3DInstance.reload(json[id], {opacity: 1});
            newActive = id;
          }                    
        });
        
        active = newActive;
      
    }, 2000);
    
    
    """))

In [ ]:
# size in html
os.stat("point_cloud_advanced.html").st_size / 1024 / 1024 # size in MB

In [ ]:
IFrame('point_cloud_advanced.html', width=900, height=350)

In [ ]:
# ratio
os.stat("point_cloud_advanced.html").st_size / os.stat("point_cloud_full.html").st_size

In [ ]:
# ratio cannot be 0.5 because we pass information about each points color